In [5]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib as mplt
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from IPython.core.interactiveshell import InteractiveShell
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import explained_variance_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
%matplotlib inline
import seaborn as sns
import math
import gc
import ipaddress

import sys
from pathlib import Path
d = Path().resolve().parent.parent
sys.path.insert(0, str(d))
import util.utils as utils
import util.plot_utils as plot_utils



plt.rcParams["figure.figsize"] = (12,4)

from xgboost import XGBClassifier
import xgboost as xgb
from sklearn.metrics import accuracy_score


InteractiveShell.ast_node_interactivity = "all"
np.set_printoptions(precision=2)

In [6]:
project_name="click-prediction"
def get_file_location(filename):
    return "../data/%s/%s" % (project_name,filename)

In [8]:
df_conv_status = pd.read_csv("../data/%s/%s" % (project_name,"train_true-step-3.csv"))
df_fraud= pd.read_csv("../data/%s/%s" % (project_name,"train_fraud-step-3.csv"))
# df_country= pd.read_csv("../data/%s/%s" % (project_name,"train_country-step-3.csv"))
# df_carrier= pd.read_csv("../data/%s/%s" % (project_name,"train_carrier-step-3.csv"))
# df_traffic= pd.read_csv("../data/%s/%s" % (project_name,"train_traffic-step-3.csv"))
# df_device= pd.read_csv("../data/%s/%s" % (project_name,"train_device-step-3.csv"))
# df_browser= pd.read_csv("../data/%s/%s" % (project_name,"train_browser-step-3.csv"))
# df_os= pd.read_csv("../data/%s/%s" % (project_name,"train_os-step-3.csv"))
#df_refurl= pd.read_csv("../data/%s/%s" % (project_name,"train_refurl-step-3.csv"))
# df_sub_pub= pd.read_csv("../data/%s/%s" % (project_name,"train_sub_pub-step-3.csv"))
# df_advert= pd.read_csv("../data/%s/%s" % (project_name,"train_advert-step-3.csv"))
df= pd.read_csv("../data/%s/%s" % (project_name,"train-step-2.csv"))

### Alternate way of adding back IP address (not encoded one)
- Way 1, copy from original df, take care of dataframe indexes and reindex if needed
- way 2, join with the encoding file that was made earlier

#### way 1

In [43]:
def fast_read_and_append(file_path,chunksize,fullsize,dtypes):
    # in chunk reading be careful as pandas might infer a columns dtype as different for diff chunk
    # As such specifying a dtype while reading by giving params to read_csv maybe better
    # Label encoding will fail if half the rows for same column is int and rest are str
    # In case of that already happened then df_test["publisherId"] = df_test["publisherId"].apply(str)
    df = pd.DataFrame()
    iterations = 0
    total_needed_iters = math.ceil(fullsize/chunksize)
    for x in pd.read_csv(file_path, chunksize=chunksize,usecols=dtypes.keys(),low_memory=False,dtype=dtypes):
        print("iterations= %s out of %s" %  (iterations,total_needed_iters))
        df = pd.concat([df, x], ignore_index=True)
        iterations += 1
    return df

dtype={'ID':np.int32,'UserIp':'str'}
# df_orig=pd.read_csv("../data/%s/%s" % (project_name,"train.csv"), sep=',', usecols=['ID','UserIp'],dtype=dtype)



In [4]:
df_orig=fast_read_and_append("../data/%s/%s" % (project_name,"train.csv"),1000000,64000000,dtype)

    

iterations= 0 out of 128
iterations= 1 out of 128
iterations= 2 out of 128
iterations= 3 out of 128
iterations= 4 out of 128
iterations= 5 out of 128
iterations= 6 out of 128
iterations= 7 out of 128
iterations= 8 out of 128
iterations= 9 out of 128
iterations= 10 out of 128
iterations= 11 out of 128
iterations= 12 out of 128
iterations= 13 out of 128
iterations= 14 out of 128
iterations= 15 out of 128
iterations= 16 out of 128
iterations= 17 out of 128
iterations= 18 out of 128
iterations= 19 out of 128
iterations= 20 out of 128
iterations= 21 out of 128
iterations= 22 out of 128
iterations= 23 out of 128
iterations= 24 out of 128
iterations= 25 out of 128
iterations= 26 out of 128
iterations= 27 out of 128
iterations= 28 out of 128
iterations= 29 out of 128
iterations= 30 out of 128
iterations= 31 out of 128
iterations= 32 out of 128
iterations= 33 out of 128
iterations= 34 out of 128
iterations= 35 out of 128
iterations= 36 out of 128
iterations= 37 out of 128
iterations= 38 out of 

KeyboardInterrupt: 

In [ ]:
df=df.sort_values("ID")
df=df.drop_duplicates(subset="ID")

df_orig_filtered=df_orig[df_orig["ID"].isin(df["ID"])]
df_orig_filtered=df_orig_filtered.sort_values("ID")
df_orig_filtered=df_orig_filtered.set_index("ID",drop=False)
# df_orig_filtered.drop(['ID'], axis = 1, inplace = True)
df=df.set_index("ID",drop=False)
df["ip"]=df_orig_filtered["UserIp"]
df.drop(['UserIp'], axis = 1, inplace = True)

In [ ]:
df_orig_cs=df_orig[df_orig["ID"].isin(df_conv_status["ID"])]
df_orig_cs=df_orig_cs.sort_values("ID")
df_orig_cs=df_orig_cs.set_index("ID",drop=False)

df_conv_status=df_conv_status.drop_duplicates(subset="ID")
df_conv_status=df_conv_status.set_index("ID",drop=False)
df_conv_status.drop(['UserIp'], axis = 1, inplace = True)
df_conv_status["ip"]=df_orig_cs["UserIp"]

In [ ]:
df_orig_fraud=df_orig[df_orig["ID"].isin(df_fraud["ID"])]
df_orig_fraud=df_orig_fraud.sort_values("ID")
df_orig_fraud=df_orig_fraud.set_index("ID",drop=False)

df_fraud=df_fraud.drop_duplicates(subset="ID")
df_fraud=df_fraud.set_index("ID",drop=False)
df_fraud.drop(['UserIp'], axis = 1, inplace = True)
df_fraud["ip"]=df_orig_fraud["UserIp"]

#### Way 2

In [9]:
ip_df=pd.read_csv("../data/%s/%s" % (project_name,"UserIp-encoding.csv"))

In [10]:
def merge_with_ip(df,ip_df):
    df_merge=pd.merge(df,ip_df,left_on="UserIp",right_on="id",sort=False)
    df_merge.drop(['id'], axis = 1, inplace = True)
    df_merge.drop(['UserIp_x'], axis = 1, inplace = True)
    df_merge["ip"] = df_merge["UserIp_y"]
    df_merge.drop(['UserIp_y'], axis = 1, inplace = True)
    return df_merge
    

In [11]:
df=merge_with_ip(df,ip_df)
df_conv_status=merge_with_ip(df_conv_status,ip_df)
df_fraud=merge_with_ip(df_fraud,ip_df)


### Adding network and subnet

In [12]:
df["network"] = df["ip"].map(lambda x: ".".join(x.split(".")[:2]))
df["subnet"] = df["ip"].map(lambda x: ".".join(x.split(".")[:3]))
df_conv_status["network"] = df_conv_status["ip"].map(lambda x: ".".join(x.split(".")[:2]))
df_conv_status["subnet"] = df_conv_status["ip"].map(lambda x: ".".join(x.split(".")[:3]))
df_fraud["network"] = df_fraud["ip"].map(lambda x: ".".join(x.split(".")[:2]))
df_fraud["subnet"] = df_fraud["ip"].map(lambda x: ".".join(x.split(".")[:3]))


In [13]:
df.drop(['ip'], axis = 1, inplace = True)

In [14]:



df_conv_status.drop(['ip'], axis = 1, inplace = True)

In [15]:



df_fraud.drop(['ip'], axis = 1, inplace = True)

In [42]:
df.to_csv("train_4.csv",index=False)
df_conv_status.to_csv("train-true_4.csv",index=False)
df_fraud.to_csv("train-fraud_4.csv",index=False)

In [6]:
# way 1 of getting test ip
df_orig=fast_read_and_append("../data/%s/%s" % (project_name,"test.csv"),500000,26000000,dtype)

iterations= 0 out of 128
iterations= 1 out of 128
iterations= 2 out of 128
iterations= 3 out of 128
iterations= 4 out of 128
iterations= 5 out of 128
iterations= 6 out of 128
iterations= 7 out of 128
iterations= 8 out of 128
iterations= 9 out of 128
iterations= 10 out of 128
iterations= 11 out of 128
iterations= 12 out of 128
iterations= 13 out of 128
iterations= 14 out of 128
iterations= 15 out of 128
iterations= 16 out of 128
iterations= 17 out of 128
iterations= 18 out of 128
iterations= 19 out of 128
iterations= 20 out of 128
iterations= 21 out of 128
iterations= 22 out of 128
iterations= 23 out of 128
iterations= 24 out of 128
iterations= 25 out of 128
iterations= 26 out of 128
iterations= 27 out of 128
iterations= 28 out of 128
iterations= 29 out of 128
iterations= 30 out of 128
iterations= 31 out of 128
iterations= 32 out of 128
iterations= 33 out of 128
iterations= 34 out of 128
iterations= 35 out of 128
iterations= 36 out of 128
iterations= 37 out of 128
iterations= 38 out of 

In [44]:
def fast_read_and_append(file_path,chunksize,fullsize):
    # in chunk reading be careful as pandas might infer a columns dtype as different for diff chunk
    # As such specifying a dtype while reading by giving params to read_csv maybe better
    # Label encoding will fail if half the rows for same column is int and rest are str
    # In case of that already happened then df_test["publisherId"] = df_test["publisherId"].apply(str)
    df = pd.DataFrame()
    iterations = 0
    total_needed_iters = math.ceil(fullsize/chunksize)
    for x in pd.read_csv(file_path, chunksize=chunksize,low_memory=False):
        print("iterations= %s out of %s" %  (iterations,total_needed_iters))
        df = pd.concat([df, x], ignore_index=True)
        iterations += 1
        gc.collect()
    return df

In [17]:
# df_test=fast_read_and_append("../data/%s/test-step-2.csv" % project_name,1000000,24510750)
df_test= pd.read_csv("../data/%s/test-step-2.csv" % project_name)

In [9]:
#way 1 of getting ip
df_orig_test=df_orig[df_orig["ID"].isin(df_test["ID"])]
df_orig_test=df_orig_test.sort_values("ID")
df_orig_test=df_orig_test.set_index("ID",drop=False)
print("===")

df_test=df_test.drop_duplicates(subset="ID")
df_test=df_test.set_index("ID",drop=False)
df_test.drop(['UserIp'], axis = 1, inplace = True)
df_test["ip"]=df_orig_test["UserIp"]


===


In [18]:
df_test=merge_with_ip(df_test,ip_df)

df_test["network"] = df_test["ip"].map(lambda x: ".".join(x.split(".")[:2]))
df_test["subnet"] = df_test["ip"].map(lambda x: ".".join(x.split(".")[:3]))
df_test.drop(['ip'], axis = 1, inplace = True)
#df_test.drop(['RefererUrl'], axis = 1, inplace = True)


In [19]:
def label_encode_field(df1,df2,df3,df_test,field):
    df1[field] = df1[field].fillna('-997')
    df2[field] = df2[field].fillna('-997')
    df3[field] = df3[field].fillna('-997')
    
    df_test[field] = df_test[field].fillna('-997')
    
    encoded_df = df1[field].append(df2[field]).append(df3[field]).append(df_test[field])
    encoded_df.fillna('-997')
    label_encoder = LabelEncoder()
    encoder = label_encoder.fit(encoded_df.values)
    encoded_t=encoder.transform(df_test[field].fillna("-997").values)
    df_test[field] = encoded_t
    
    encoded_df1=encoder.transform(df1[field].fillna("-997").values)
    df1[field] = encoded_df1
    
    encoded_df2=encoder.transform(df2[field].fillna("-997").values)
    df2[field] = encoded_df2
    
    encoded_df3=encoder.transform(df3[field].fillna("-997").values)
    df3[field] = encoded_df3
    
    return encoder


def store_encoder_as_file(le,column_name):
    ids =np.arange(0,len(list(le.classes_))).astype(int)
    my_encoding = pd.DataFrame(list(le.classes_), ids, columns = [column_name])
    print(my_encoding.shape)
    my_encoding.to_csv("%s-encoding.csv" % column_name, index_label = ["id"])

In [26]:
# Dont use this since we will need refurl later
# dtype={"network":"object"}
# df_conv_status = pd.read_csv("train-true_4.csv",dtype=dtype)
# df_fraud= pd.read_csv("train-fraud_4.csv",dtype=dtype)

# df= pd.read_csv("train_4.csv",dtype=dtype)

# df_conv_status.drop(['RefererUrl'], axis = 1, inplace = True)
# df_fraud.drop(['RefererUrl'], axis = 1, inplace = True)
# df.drop(['RefererUrl'], axis = 1, inplace = True)



In [20]:
store_encoder_as_file(label_encode_field(df_conv_status,df_fraud,df,df_test,"network"),"network-step-4")
store_encoder_as_file(label_encode_field(df_conv_status,df_fraud,df,df_test,"subnet"),"subnet-step-4")



(36597, 1)
(1745892, 1)


In [21]:
df_test.to_csv("test_5.csv",index=False)

In [22]:
df.to_csv("train_5.csv",index=False)
df_conv_status.to_csv("train-true_5.csv",index=False)
df_fraud.to_csv("train-fraud_5.csv",index=False)